# Perform some queries 

After you've loaded your database in your RDBMS, the priority is to solve all the business questions using SQL queries instead of Pandas dataframe commands:

1. What is the category generating the maximum sales revenue?

    - What about the profit in this category?
    - Are they making a loss in any categories?
2. What are 5 states generating the maximum and minimum sales revenue?

3. What are the 3 products in each product segment with the highest sales?

    - Are they the 3 most profitable products as well?
4. What are the 3 best-seller products in each product segment? (Quantity-wise)

5. What are the top 3 worst-selling products in every category? (Quantity-wise)

6. How many unique customers per month are there for the year 2016. (There's a catch here: contrary to other 'heavier' RDBMS, SQLite does not support the functions YEAR() or MONTH() to extract the year or the month in a date. You will have to create two new columns: year and month.)

In [2]:
# Import libraries 
import sqlite3
import pandas as pd
import plotly.express as px

In [3]:
# connecting to the database
connection = sqlite3.connect("DB_browser/SuperStore.db")

print("Connected to the database")

Connected to the database


## Functions:

In [4]:
# Display the query in a dataframe
def dataframe(query):
    df = pd.read_sql(query, connection)
    return df

### Category by Total Sales 

In [5]:
category_sales = '''   
                    SELECT c.Category, ROUND(SUM(o.Sales),2)'TotalSales'
                    FROM Categories AS c
                    JOIN Products AS p
                    ON c.CategoryID = p.CategoryId
                    JOIN Orders_details AS o
                    ON p.ProductId = o.ProductId
                    GROUP BY c.Category;
                '''


df = dataframe(category_sales)

# Add percentage column for each Category
df['Percentage'] = round((df['TotalSales'] / df['TotalSales'].sum() * 100),2)

# Plot the data
fig = px.bar(df, x='Category', y='TotalSales', title='Total Sales by Category', 
             template='plotly_dark', color='TotalSales', color_continuous_scale=px.colors.sequential.Viridis, 
             text='Percentage')

# Add percentage to the hover text
fig.update_traces(texttemplate='%{text}%')


fig.show()


### Category by Total Profits

In [6]:
category_profits = '''   
                    SELECT c.Category, ROUND(SUM(o.Profit),2)'TotalProfits'
                    FROM Categories AS c
                    JOIN Products AS p
                    ON c.CategoryID = p.CategoryId
                    JOIN Orders_details AS o
                    ON p.ProductId = o.ProductId
                    GROUP BY c.Category;
                '''

df = dataframe(category_profits)
# Plot the data
fig = px.pie(df, names='Category', values='TotalProfits', title='Total Profits by Category', template='plotly_dark')
fig.show()

### What is the category generating the maximum sales revenue?

In [7]:
category_max_sales = ''' 
                        WITH
                        SalesCTE AS (
                            SELECT c.Category, ROUND(SUM(o.Sales), 2)'TotalSales'
                            FROM Categories AS c
                            JOIN Products AS p
                            ON c.CategoryID = p.CategoryId
                            JOIN Orders_details AS o
                            ON p.ProductId = o.ProductId
                            GROUP BY c.Category
                        )
                        SELECT *
                        FROM SalesCTE
                        WHERE TotalSales IN (
                            SELECT MAX(TotalSales)
                            FROM SalesCTE                
                                            );
                    '''


dataframe(category_max_sales)


,Category,TotalSales
0,Technology,836154.03


#### What about the profit in this category?

In [8]:
category_tech_profits = '''   
                    SELECT c.Category, ROUND(SUM(o.Profit),2)'TotalProfits'
                    FROM Categories AS c
                    JOIN Products AS p
                    ON c.CategoryID = p.CategoryId
                    JOIN Orders_details AS o
                    ON p.ProductId = o.ProductId
                    WHERE c.Category = 'Technology'
                    GROUP BY c.Category;
                '''
                
dataframe(category_tech_profits)

,Category,TotalProfits
0,Technology,145454.95


#### Are they making a loss in any categories?

In [9]:
category_make_loss = '''   
                    SELECT c.Category, ROUND(SUM(o.Profit),2)'TotalProfits'
                    FROM Categories AS c
                    JOIN Products AS p
                    ON c.CategoryID = p.CategoryId
                    JOIN Orders_details AS o
                    ON p.ProductId = o.ProductId
                    WHERE o.Profit < 0
                    GROUP BY c.Category;
                '''
                
dataframe(category_make_loss)

,Category,TotalProfits
0,Furniture,-60936.11
1,Office Supplies,-56615.26
2,Technology,-38579.92


### What are 5 states generating the maximum and minimum sales revenue?

#### 5 States generating the maximum sales revenue:

In [20]:
states_max_sales = '''   
                    SELECT a.State, ROUND(SUM(o.SALES), 2)'TotalSales'
                    FROM Orders_details AS o
                    JOIN Shipments AS s
                    ON o.RowId = s.RowId
                    JOIN Address AS a
                    ON s.AddressId = a.AddressID
                    GROUP BY a.State
                    ORDER BY 
                        TotalSales DESC
                    LIMIT 5;
                '''
                
dataframe(states_max_sales)

,State,TotalSales
0,California,457687.63
1,New York,310876.27
2,Texas,170188.05
3,Washington,138641.27
4,Pennsylvania,116511.91


#### 5 States generating the minimun sales revenue:

In [21]:
states_min_sales = '''   
                    SELECT a.State, ROUND(SUM(o.SALES), 2)'TotalSales'
                    FROM Orders_details AS o
                    JOIN Shipments AS s
                    ON o.RowId = s.RowId
                    JOIN Address AS a
                    ON s.AddressId = a.AddressID
                    GROUP BY a.State
                    ORDER BY 
                        TotalSales ASC
                    LIMIT 5;
                '''
                
dataframe(states_min_sales)

,State,TotalSales
0,North Dakota,919.91
1,West Virginia,1209.82
2,Maine,1270.53
3,South Dakota,1315.56
4,Wyoming,1603.14


### What are the 3 products in each product segment with the highest sales?

In [40]:
product_segment_high_sales = '''   
                            WITH
                            RankProducts AS (
                                SELECT 
                                    c.SubCategory, p.ProductName, ROUND(SUM(o.Sales))'TotalSales',
                                    RANK() OVER (PARTITION BY c.SubCategory ORDER BY SUM(o.Sales) DESC)'SalesRank'
                                FROM Orders_details AS o
                                JOIN Products AS p
                                ON o.ProductId = p.ProductId
                                JOIN Categories AS c
                                ON p.CategoryId = c.CategoryId
                                GROUP BY p.ProductName
                            )
                            SELECT *
                            FROM RankProducts
                            WHERE SalesRank <= 3;
                        '''
                
dataframe(product_segment_high_sales)

,SubCategory,ProductName,TotalSales,SalesRank
0,Accessories,Logitech G19 Programmable Gaming Keyboard,13757.0,1
1,Accessories,Logitech P710e Mobile Speakerphone,11204.0,2
2,Accessories,Plantronics CS510 - Over-the-Head monaural Wir...,10822.0,3
3,Appliances,Honeywell Enviracaire Portable HEPA Air Cleane...,11304.0,1
4,Appliances,Hoover Upright Vacuum With Dirt Cup,6833.0,2
5,Appliances,Sanyo Counter Height Refrigerator with Crisper...,5907.0,3
6,Art,"Hunt PowerHouse Electric Pencil Sharpener, Blue",1618.0,1
7,Art,Boston Heavy-Duty Trimline Electric Pencil Sha...,1166.0,2
8,Art,Hunt BOSTON Model 1606 High-Volume Electric Pe...,1113.0,3
9,Binders,Fellowes PB500 Electric Punch Plastic Comb Bin...,27453.0,1


#### Are they the 3 most profitable products as well?

In [47]:
product_segment_high_sales_profit = '''   
                            WITH
                            RankProducts AS (
                                SELECT 
                                    c.SubCategory, p.ProductName, ROUND(SUM(o.Sales))'TotalSales', ROUND(SUM(o.Profit))'TotalProfits',
                                    RANK() OVER (PARTITION BY c.SubCategory ORDER BY SUM(o.Sales) DESC, SUM(o.Profit) DESC)'SalesProfitRank'
                                FROM Orders_details AS o
                                JOIN Products AS p
                                ON o.ProductId = p.ProductId
                                JOIN Categories AS c
                                ON p.CategoryId = c.CategoryId
                                GROUP BY p.ProductName
                            )
                            SELECT *,
                            CASE 
                                WHEN TotalProfits > 0 THEN 'Yes'
                                ELSE 'No'
                            END AS Profitable
                            FROM RankProducts
                            WHERE SalesProfitRank <= 3;
                        '''
                
dataframe(product_segment_high_sales_profit)

,SubCategory,ProductName,TotalSales,TotalProfits,SalesProfitRank,Profitable
0,Accessories,Logitech G19 Programmable Gaming Keyboard,13757.0,4425.0,1,Yes
1,Accessories,Logitech P710e Mobile Speakerphone,11204.0,1988.0,2,Yes
2,Accessories,Plantronics CS510 - Over-the-Head monaural Wir...,10822.0,3085.0,3,Yes
3,Appliances,Honeywell Enviracaire Portable HEPA Air Cleane...,11304.0,3247.0,1,Yes
4,Appliances,Hoover Upright Vacuum With Dirt Cup,6833.0,460.0,2,Yes
5,Appliances,Sanyo Counter Height Refrigerator with Crisper...,5907.0,1654.0,3,Yes
6,Art,"Hunt PowerHouse Electric Pencil Sharpener, Blue",1618.0,448.0,1,Yes
7,Art,Boston Heavy-Duty Trimline Electric Pencil Sha...,1166.0,311.0,2,Yes
8,Art,Hunt BOSTON Model 1606 High-Volume Electric Pe...,1113.0,111.0,3,Yes
9,Binders,Fellowes PB500 Electric Punch Plastic Comb Bin...,27453.0,7753.0,1,Yes


### What are the 3 best-seller products in each product segment? (Quantity-wise)

In [73]:
product_segment_best_quantity = '''  
                                WITH RankQuantity AS (
                                    SELECT 
                                        c.SubCategory, p.ProductName, SUM(o.Quantity)'TotalQuantity',
                                        ROW_NUMBER() OVER (PARTITION BY c.SubCategory ORDER BY SUM(QUANTITY) DESC)'TotalQuantityRank'
                                    FROM Orders_details AS o
                                    JOIN Products AS p
                                    ON o.ProductId = p.ProductId
                                    JOIN Categories AS c
                                    ON p.CategoryId = c.CategoryId
                                    GROUP BY p.ProductName
                                )
                                SELECT *
                                FROM RankQuantity
                                WHERE TotalQuantityRank <= 3;
                                '''
                        
dataframe(product_segment_best_quantity)

,SubCategory,ProductName,TotalQuantity,TotalQuantityRank
0,Accessories,Logitech P710e Mobile Speakerphone,75,1
1,Accessories,Logitech G19 Programmable Gaming Keyboard,60,2
2,Accessories,Kingston Digital DataTraveler 16GB USB 2.0,57,3
3,Appliances,Staple holder,52,1
4,Appliances,"Acco Six-Outlet Power Strip, 4' Cord Length",41,2
5,Appliances,Fellowes Basic Home/Office Series Surge Protec...,41,3
6,Art,Staples in misc. colors,86,1
7,Art,Newell 312,49,2
8,Art,4009 Highlighters by Sanford,44,3
9,Binders,Avery Non-Stick Binders,71,1


### What are the top 3 worst-selling products in every category? (Quantity-wise)

In [72]:
product_category_worst_quantity = '''
                                WITH RankQuantity AS (
                                    SELECT 
                                        c.Category, p.ProductName, SUM(o.Quantity)'TotalQuantity',
                                        ROW_NUMBER() OVER (PARTITION BY c.Category ORDER BY SUM(o.Quantity) ASC)'TotalQuantityRank'
                                    FROM Orders_details AS o
                                    JOIN Products AS p
                                    ON o.ProductId = p.ProductId
                                    JOIN Categories AS c
                                    ON p.CategoryId = c.CategoryId
                                    GROUP BY p.ProductName
                                )
                                SELECT *
                                FROM RankQuantity
                                WHERE TotalQuantityRank <= 3;
                                '''
                                
dataframe(product_category_worst_quantity)

,Category,ProductName,TotalQuantity,TotalQuantityRank
0,Furniture,"Bush Saratoga Collection 5-Shelf Bookcase, Han...",1,1
1,Furniture,Global Enterprise Series Seating Low-Back Swiv...,1,2
2,Furniture,Barricks Non-Folding Utility Table with Steel ...,2,3
3,Office Supplies,Boston 1900 Electric Pencil Sharpener,1,1
4,Office Supplies,Xerox 20,1,2
5,Office Supplies,Avery 5,2,3
6,Technology,Penpower WorldCard Pro Card Scanner,1,1
7,Technology,Canon imageCLASS MF7460 Monochrome Digital Las...,2,2
8,Technology,Hewlett-Packard Deskjet F4180 All-in-One Color...,2,3


### How many unique customers per month are there for the year 2016. (There's a catch here: contrary to other 'heavier' RDBMS, SQLite does not support the functions YEAR() or MONTH() to extract the year or the month in a date. You will have to create two new columns: year and month.)

In [77]:
unique_customer_month_2016 = '''
                                SELECT
                                    STRFTIME('%Y', OrderDate)'Year',
                                    STRFTIME('%m', OrderDate)'Month',
                                    COUNT(DISTINCT CustomerId)'UniqueCustomers'
                                FROM Orders_details AS od
                                JOIN Orders AS o
                                ON od.OrderId = o.OrderId
                                WHERE STRFTIME('%Y', OrderDate) = '2016'
                                GROUP BY Year, Month;
                            '''
                            
dataframe(unique_customer_month_2016)

,Year,Month,UniqueCustomers
0,2016,01,46
1,2016,02,42
2,2016,03,80
3,2016,04,83
4,2016,05,96
5,2016,06,90
6,2016,07,89
7,2016,08,86
8,2016,09,176
9,2016,10,95


#### Number of unique Customer par month in 2016:

In [82]:
unique_customer_month_2016 = '''
                                SELECT
                                    STRFTIME('%Y', OrderDate)'Year',
                                    STRFTIME('%m', OrderDate)'Month',
                                    COUNT(DISTINCT CustomerId)'UniqueCustomers',
                                    OrderDate
                                FROM Orders_details AS od
                                JOIN Orders AS o
                                ON od.OrderId = o.OrderId
                                WHERE STRFTIME('%Y', OrderDate) = '2016'
                                GROUP BY Year, Month;
                            '''
                            

df = dataframe(unique_customer_month_2016)

fig = px.line(df, x='OrderDate', y='UniqueCustomers', title='Unique Customers per Month in 2016')
fig.show()